In [14]:
import torch
import torch.nn as nn
import torch.nn.init as init
net = nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))#pytorch已经默认初始化
print(net)
X = torch.rand(2,4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [4]:
X = torch.rand(2,4)
Y = net(X).sum()

In [5]:
Y

tensor(0.8288, grad_fn=<SumBackward0>)

In [6]:
#访问模型参数
print(type(net.named_parameters()))

<class 'generator'>


In [7]:
for name,param in net.named_parameters():
    print(name,param.size())

0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [8]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


另外返回的param的类型为torch.nn.parameter.Parameter，其实这是Tensor的子类，和Tensor不同的是如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里，来看下面这个例子。

In [10]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.weight1 = nn.Parameter(torch.rand(20, 20))#自动添加进参数列表
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [11]:
weight_0 = list(net[0].parameters())[0]

In [12]:
weight_0

Parameter containing:
tensor([[-0.2189,  0.3182,  0.0961,  0.2372],
        [ 0.1310, -0.3185,  0.4021, -0.2052],
        [ 0.3823, -0.1872,  0.0172,  0.4445]], requires_grad=True)

In [13]:
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

tensor([[-0.2189,  0.3182,  0.0961,  0.2372],
        [ 0.1310, -0.3185,  0.4021, -0.2052],
        [ 0.3823, -0.1872,  0.0172,  0.4445]])
None
tensor([[0.0466, 0.0207, 0.0453, 0.0108],
        [0.2644, 0.1316, 0.2604, 0.0387],
        [0.3167, 0.1408, 0.3081, 0.0736]])


In [15]:
#初始化模型参数
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[-0.0042, -0.0040, -0.0158, -0.0064],
        [ 0.0028,  0.0145,  0.0083,  0.0085],
        [ 0.0002,  0.0029,  0.0245, -0.0081]])
2.weight tensor([[ 0.0087, -0.0102,  0.0007]])


In [16]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


In [17]:
#自定义初始化方法
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor *= (tensor.abs() >=5).float()

In [18]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 0.0000, -8.2221,  8.2984, -7.3519],
        [ 0.0000,  6.8923,  0.0000, -5.5485],
        [ 9.4525,  5.8482,  0.0000,  6.2381]])
2.weight tensor([[-0.0000, -7.7434, -6.0079]])


我们还可以通过改变这些参数的data来改写模型参数值同时不会影响梯度:

In [19]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


In [20]:
#共享模型参数
#Module类的forward函数里多次调用同一个层。此外，如果我们传入Sequential的模块是同一个Module实例的话参数也是共享的
linear = nn.Linear(1,1,bias=False)
net = nn.Sequential(linear,linear)
print(net)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)


In [21]:
for name,param in net.named_parameters():
    init.constant_(param,val=3)
    print(name,param.data)

0.weight tensor([[3.]])


In [22]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))#在内存中，这两个线形层其实是一个对象

True
True


In [23]:
#因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) # 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
